In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/MyDrive/code/inactivation")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/code/inactivation'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.rcParams['font.sans-serif'] = ['Arial'] 
from scipy import stats
import pickle
import math

In [3]:
data = pd.read_csv("data/bayesian_fitting.csv")
TIME = data['TIME']
Nt = data['Nt']
Initial = data['Initial']

In [4]:
#parameter estimation
#compile
import pystan
from pystan import StanModel

stan_code = """
  data {
    int M;
	  int<lower=0> Nt[M];
	  real TIME[M];
	  int Initial[M];
  }

  parameters {
	  real<lower=0> a;
	  real<lower=0> b;  
	  real<lower=0> n0;
  }

  model {
	  for (m in 1:M) {
		  if (Initial[m] != 0 ) {
			  Nt[Initial[m]] ~ poisson(n0);
		  }
	  }
    for (m in 1:M) {        
        Nt[m] ~ poisson(n0*(10^(-((TIME[m]/a)^b))));
    }
  }

"""

model = pystan.StanModel(model_code=stan_code)
with open('data/model-inactivation.pkl','wb') as f:
    pickle.dump(model,f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_91e034ba85fd6fc7ba4b9224dc42f550 NOW.


In [5]:
stan_data = data.to_dict('list')
stan_data.update({'M':len(data)})

In [6]:
#MCMC sampling
fit_nuts = model.sampling(data = stan_data,iter=10000,chains=4,warmup=5000)
with open('data/result-inactivation.pkl','wb') as g:
    pickle.dump(fit_nuts,g)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  after removing the cwd from sys.path.


In [7]:
fit_nuts

Inference for Stan model: anon_model_91e034ba85fd6fc7ba4b9224dc42f550.
4 chains, each with iter=10000; warmup=5000; thin=1; 
post-warmup draws per chain=5000, total post-warmup draws=20000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a      1.59  1.8e-5 2.2e-3   1.59   1.59   1.59   1.59   1.59  15072    1.0
b      1.21  2.9e-5 3.2e-3    1.2    1.2   1.21   1.21   1.21  12187    1.0
n0    8.4e4    0.97 116.94  8.3e4  8.4e4  8.4e4  8.4e4  8.4e4  14539    1.0
lp__  6.7e6    0.01   1.22  6.7e6  6.7e6  6.7e6  6.7e6  6.7e6  10418    1.0

Samples were drawn using NUTS at Mon Mar  8 00:52:14 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).